<a href="https://colab.research.google.com/github/jdluebbe/MLB_Pitch_Prediction/blob/master/Pitch_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib as plt
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('./DRIVE/', force_remount = True)

Mounted at ./DRIVE/


Firstly, I must decide how to approach this problem. In a quick scan through the academic literature I came across this paper "Using Multi-Class Classification Methods to Predict Baseball Pitch Types" by Glenn Silde. In this paper he and his co-author explore several possible methods of predicting the next pitch a pitcher may throw. They applied multi-class SVM's, a neural network, random forest, and linear discriminate analysis. They're best performing model was their random forests models so this is route I chose to pursue.

In [3]:
pitch = pd.read_csv('./DRIVE/My Drive/Pitches/pitches')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pitch.head(10)

,uid,game_pk,year,date,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,...,runner7_start,runner7_end,runner7_event,runner7_score,runner7_rbi,runner7_earned,created_at,added_at,modified_at,modified_by
0,14143226,286874,2011,2011-03-31,108,118,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
1,14143227,286874,2011,2011-03-31,108,118,1,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
2,14143228,286874,2011,2011-03-31,108,118,1,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
3,14143229,286874,2011,2011-03-31,108,118,1,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
4,14143230,286874,2011,2011-03-31,108,118,1,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
5,14143231,286874,2011,2011-03-31,108,118,1,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
6,14143232,286874,2011,2011-03-31,108,118,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
7,14143233,286874,2011,2011-03-31,108,118,1,1,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
8,14143234,286874,2011,2011-03-31,108,118,1,1,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1
9,14143235,286874,2011,2011-03-31,108,118,1,1,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 21:33:20,2016-03-03 21:33:20,2016-03-03 21:33:20,1


In [5]:
pitch.describe()

,uid,game_pk,year,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,pcount_pitcher,...,runner6_rbi,runner6_earned,runner7_id,runner7_start,runner7_end,runner7_event,runner7_score,runner7_rbi,runner7_earned,modified_by
count,7.189610e+05,718961.000000,718961.0,718961.000000,718961.000000,718961.000000,718961.000000,718961.000000,718961.000000,718961.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,718961.0
mean,1.701980e+07,288557.611823,2011.0,128.787182,128.757296,5.033796,0.508900,39.301513,2.865276,37.357196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
std,1.665143e+06,3642.210956,0.0,14.281039,14.306711,2.692326,0.499921,23.320725,1.715068,30.690569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,1.414323e+07,286874.000000,2011.0,108.000000,108.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25%,1.557356e+07,287507.000000,2011.0,115.000000,115.000000,3.000000,0.000000,19.000000,1.000000,11.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
50%,1.703106e+07,288129.000000,2011.0,134.000000,134.000000,5.000000,1.000000,39.000000,3.000000,28.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
75%,1.845894e+07,288749.000000,2011.0,141.000000,141.000000,7.000000,1.000000,58.000000,4.000000,61.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
max,1.983820e+07,317073.000000,2011.0,158.000000,158.000000,19.000000,1.000000,158.000000,16.000000,133.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [6]:
pitch.shape

(718961, 125)

In [7]:
pitch.tail(10)

,uid,game_pk,year,date,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,...,runner7_start,runner7_end,runner7_event,runner7_score,runner7_rbi,runner7_earned,created_at,added_at,modified_at,modified_by
718951,19838187,317073,2011,2011-10-28,140,138,9,1,71,2,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718952,19838188,317073,2011,2011-10-28,140,138,9,1,71,3,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718953,19838189,317073,2011,2011-10-28,140,138,9,1,71,4,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718954,19838190,317073,2011,2011-10-28,140,138,9,1,72,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718955,19838191,317073,2011,2011-10-28,140,138,9,1,72,2,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718956,19838192,317073,2011,2011-10-28,140,138,9,1,72,3,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718957,19838193,317073,2011,2011-10-28,140,138,9,1,72,4,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718958,19838194,317073,2011,2011-10-28,140,138,9,1,72,5,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718959,19838195,317073,2011,2011-10-28,140,138,9,1,73,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1
718960,19838196,317073,2011,2011-10-28,140,138,9,1,73,2,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-03 22:23:19,2016-03-03 22:23:19,2016-03-03 22:23:19,1


Now that I have identified the data containing the labels that I wish to predict. I will clean up the remaining data by dropping Null values, and columns unrelated to pitch prediction.

In [8]:
pitch['pitch_type'][100:110]

100    FF
101    FF
102    FF
103    SI
104    CH
105    FF
106    SI
107    FC
108    FC
109    FF
Name: pitch_type, dtype: object

Since I will be using pitch_type as my label any rows that are missing a value will be useless for prediction, so I'm going to drop those rows from the dataset.

In [0]:
pitch = pitch.dropna(subset=['pitch_type'])

In [10]:
print(pitch['pitch_type'].isna().any())
print(len(pitch['pitch_type']))

False
716681


In [0]:
pitch_clean = pitch.drop(
    ['runner1_id', 'runner1_start', 'runner1_end', 'runner1_event', 'runner1_score',
     'runner1_rbi', 'runner1_earned', 'runner2_id', 'runner2_start', 'runner2_end', 'runner2_event', 'runner2_score', 'runner2_rbi', 'runner2_earned',
     'runner3_id', 'runner3_start', 'runner3_end', 'runner3_event', 'runner3_score', 'runner3_rbi', 'runner3_earned', 'runner4_id', 'runner4_start', 'runner4_end',
     'runner4_event', 'runner4_score', 'runner4_rbi',
     'runner4_earned', 'runner5_id', 'runner5_start', 'runner5_end', 'runner5_event', 'runner5_score', 'runner5_rbi', 'runner5_earned', 'runner6_id', 'runner6_start',
     'runner6_end', 'runner6_event', 'runner6_score', 'on_1b', 'on_2b', 'on_3b', 'runner6_rbi', 'runner6_earned', 'runner7_id', 'runner7_start', 'runner7_end', 'runner7_event',
     'runner7_score',
     'runner7_rbi', 'runner7_earned', 'pitch_tfs_zulu', 'cc', 'pitch_type', 'sv_id', 'date', 'b_height', 'pitch_tfs', 'start_tfs_zulu', 'at_bat_des', 'created_at', 'added_at', 'modified_at',
     'event2', 'event3', 'event4', 'score', 'uid', 'game_pk'], axis=1)

In [12]:
pitch_clean.head(3)

,year,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,pcount_pitcher,balls,strikes,...,ay,break_length,break_y,break_angle,type_confidence,zone,nasty,spin_dir,spin_rate,modified_by
26,2011,118,108,1,0,7,1,1,0,0,...,22.579,2.8,23.9,-0.7,2.0,8.0,47.0,183.148,2519.455,1
27,2011,118,108,1,0,7,2,2,0,1,...,26.928,1.9,23.8,6.9,2.0,12.0,25.0,187.663,2838.803,1
28,2011,118,108,1,0,7,3,3,1,1,...,24.831,2.3,23.9,-12.4,2.0,14.0,64.0,179.643,2701.919,1


While that may look like I am throwing away a large portion of the dataset by dropping all of these columns, many of these columns were almost completely null values. The remainder were either so sparse they contributed very little data or were in my view unrelated to the current prediction task. 

In [13]:
pitch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716681 entries, 26 to 718960
Data columns (total 55 columns):
year               716681 non-null int64
team_id_b          716681 non-null int64
team_id_p          716681 non-null int64
inning             716681 non-null int64
top                716681 non-null int64
at_bat_num         716681 non-null int64
pcount_at_bat      716681 non-null int64
pcount_pitcher     716681 non-null int64
balls              716681 non-null int64
strikes            716681 non-null int64
fouls              716681 non-null int64
outs               716681 non-null int64
is_final_pitch     716681 non-null int64
final_balls        716681 non-null int64
final_strikes      716681 non-null int64
final_outs         716681 non-null int64
start_tfs          716681 non-null int64
batter_id          716681 non-null int64
stand              716681 non-null object
pitcher_id         716681 non-null int64
p_throws           716681 non-null object
event              716681

In [14]:
pitch_clean.shape

(716681, 55)

The random forest cannot run with several of the columns being represented as categorical objects. While it would be convenient to simpy discard these columns like I did with columns above, but in my view these columns do contain useful and relevant information so I would like to preserve it. To circumvent this fact I will utilize hot encoding to convert these objects into int's.

In [0]:
from sklearn.preprocessing import LabelBinarizer
type_lb = LabelBinarizer()
stand_lb = LabelBinarizer()
p_throws_lb = LabelBinarizer()
pitch_des_lb = LabelBinarizer()
event_lb = LabelBinarizer()
#pitch_type_lb = LabelBinarizer()

L_type = type_lb.fit_transform(pitch_clean.type.values)
L_stand = stand_lb.fit_transform(pitch_clean.stand.values)
L_p_throws = p_throws_lb.fit_transform(pitch_clean.p_throws.values)
L_pitch_des = pitch_des_lb.fit_transform(pitch_clean.pitch_des.values)
L_event = event_lb.fit_transform(pitch_clean.event.values)
#L_pitch_type = pitch_type_lb.fit_transform(pitch.pitch_type.values)

In [16]:
print(L_type[0:10])
#print(L_stand)
#print(L_p_throws)
#print(L_pitch_des)
#print(L_event)


[[0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]]


In [17]:
print(L_type.shape)
print(L_stand.shape)
print(L_p_throws.shape)
print(L_pitch_des.shape)
print(L_event.shape)


(716681, 3)
(716681, 1)
(716681, 1)
(716681, 18)
(716681, 30)


Using LabelBinarizer I have replaced the categorical (object type) data into columns of binary integers, this will preserve the information of these columns while still allowing them to be processed by the random forest. Now I will drop the original columns from the dataset and replace them with the numpy arrays generated by LabelBinarizer. I'll use the function I wrote for the larger numpy arrays.

In [0]:
pitch_pure = pitch_clean.drop(['type', 'stand', 'p_throws', 'pitch_des', 'event', 'nasty'], axis=1)


In [0]:
pitch_pure['L_type_0'] = L_type[:, 0]
pitch_pure['L_type_1'] = L_type[:, 1]
pitch_pure['L_type_2'] = L_type[:, 2]
pitch_pure['L_stand'] = L_stand[:, 0]
pitch_pure['L_p_throws'] = L_p_throws[:, 0]

In [0]:
def column_creator(df, column, column_name):
  for i in range(column.shape[1]):
    df[column_name + '_' + str(i)] = column[:, i] 

In [0]:
column_creator(pitch_pure, L_pitch_des, 'L_pitch_des')
column_creator(pitch_pure, L_event, 'L_event')
#column_creator(pitch, L_pitch_type, 'L_pitch_type')

Now to make sure the columns are being properly added to the original dataset.

In [22]:
pitch_pure.columns


Index(['year', 'team_id_b', 'team_id_p', 'inning', 'top', 'at_bat_num',
       'pcount_at_bat', 'pcount_pitcher', 'balls', 'strikes',
       ...
       'L_event_20', 'L_event_21', 'L_event_22', 'L_event_23', 'L_event_24',
       'L_event_25', 'L_event_26', 'L_event_27', 'L_event_28', 'L_event_29'],
      dtype='object', length=102)

Make sure that all of the objects now either floats or int's

In [23]:
print(pitch_pure.info())
print(pitch_pure.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716681 entries, 26 to 718960
Columns: 102 entries, year to L_event_29
dtypes: float64(26), int64(76)
memory usage: 563.2 MB
None
(716681, 102)


Random Forests also do not process nan values, so I must identify which columns contain such values and then convert them into a form that is passable for the classifier, while not biasing the data too severely. Towards this purpose I chose to replace the nan values with the mean of their respective column as a 0 for each nan value would be more detrimental. 

In [24]:
pitch_pure.isnull().any()
#print(pitch_pure.isna().any()[0:50])
print(pitch_pure.isna().any()[0:102])

year              False
team_id_b         False
team_id_p         False
inning            False
top               False
at_bat_num        False
pcount_at_bat     False
pcount_pitcher    False
balls             False
strikes           False
fouls             False
outs              False
is_final_pitch    False
final_balls       False
final_strikes     False
final_outs        False
start_tfs         False
batter_id         False
pitcher_id        False
away_team_runs    False
home_team_runs    False
pitch_id          False
x                 False
y                 False
start_speed       False
end_speed         False
sz_top            False
sz_bot            False
pfx_x             False
pfx_z             False
                  ...  
L_event_0         False
L_event_1         False
L_event_2         False
L_event_3         False
L_event_4         False
L_event_5         False
L_event_6         False
L_event_7         False
L_event_8         False
L_event_9         False
L_event_10      

Imputer requires the newest version of scikit-learn



In [0]:
#!pip uninstall scikit-learn 
#!pip install Cython
#!pip install git+git://github.com/scikit-learn/scikit-learn.git

In [26]:
#!pip freeze | grep scikit
#!pip install scipy
from numpy import nan
import scipy
import sklearn
from sklearn.preprocessing import Imputer  
imp = Imputer(copy=True, missing_values=nan, strategy='mean', verbose=0)
imp.fit(pitch_pure['start_speed'].reshape(-1, 1))
pitch_pure['start_speed'] = imp.transform(pitch_pure['start_speed'].reshape(-1, 1))           
print(pitch_pure['start_speed'].isna().any())

False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [27]:
imp.fit(pitch_pure['end_speed'].reshape(-1, 1))
pitch_pure['end_speed'] = imp.transform(pitch_pure['end_speed'].reshape(-1, 1)) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


This function takes in a column converts it's nan's to means and then outputs a boolean test of the column as confirmation.

In [0]:
def nan_buster(df, column):
  imp.fit(df[column].reshape(-1, 1))
  df[column] = imp.transform(df[column].reshape(-1, 1))
  print(df[column].isna().any())

In [29]:
nan_buster(pitch_pure, 'sz_top')

False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
nan_buster(pitch_pure, 'sz_bot')
nan_buster(pitch_pure, 'pfx_x')
nan_buster(pitch_pure, 'pfx_z')
nan_buster(pitch_pure, 'px')
nan_buster(pitch_pure, 'pz')
nan_buster(pitch_pure, 'x0')
nan_buster(pitch_pure, 'z0')
nan_buster(pitch_pure, 'y0')
nan_buster(pitch_pure, 'vx0')
nan_buster(pitch_pure, 'vz0')
nan_buster(pitch_pure, 'vy0')
nan_buster(pitch_pure, 'ax')
nan_buster(pitch_pure, 'az')
nan_buster(pitch_pure, 'ay')
nan_buster(pitch_pure, 'break_length')
nan_buster(pitch_pure, 'break_y')
nan_buster(pitch_pure, 'break_angle')
nan_buster(pitch_pure, 'type_confidence')
nan_buster(pitch_pure, 'zone')
nan_buster(pitch_pure, 'spin_dir')
nan_buster(pitch_pure, 'spin_rate')

False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


Now confirm all columns sucessfully contain no nan's

In [31]:
print(pitch_pure.isna().any())

year              False
team_id_b         False
team_id_p         False
inning            False
top               False
at_bat_num        False
pcount_at_bat     False
pcount_pitcher    False
balls             False
strikes           False
fouls             False
outs              False
is_final_pitch    False
final_balls       False
final_strikes     False
final_outs        False
start_tfs         False
batter_id         False
pitcher_id        False
away_team_runs    False
home_team_runs    False
pitch_id          False
x                 False
y                 False
start_speed       False
end_speed         False
sz_top            False
sz_bot            False
pfx_x             False
pfx_z             False
                  ...  
L_event_0         False
L_event_1         False
L_event_2         False
L_event_3         False
L_event_4         False
L_event_5         False
L_event_6         False
L_event_7         False
L_event_8         False
L_event_9         False
L_event_10      

In [32]:
print(pitch_pure.columns)

Index(['year', 'team_id_b', 'team_id_p', 'inning', 'top', 'at_bat_num',
       'pcount_at_bat', 'pcount_pitcher', 'balls', 'strikes',
       ...
       'L_event_20', 'L_event_21', 'L_event_22', 'L_event_23', 'L_event_24',
       'L_event_25', 'L_event_26', 'L_event_27', 'L_event_28', 'L_event_29'],
      dtype='object', length=102)


A significant portion of the data is stored as float64, however RandomForests currently cannot handle this data type and so I converted the frame to float32

In [0]:
pitch_pure = pitch_pure.astype(np.float32)

In [34]:
pitch_pure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716681 entries, 26 to 718960
Columns: 102 entries, year to L_event_29
dtypes: float32(102)
memory usage: 284.3 MB


Below is a command used to sub-sample the dataset as I am iterating through tests of the Randomforest to make the computation proceed faster, and  then once all the code and data is properly oriented introduce the entire dataset.

In [0]:
#pitch_pure = pd.DataFrame.sample(pitch_pure, frac = .1, random_state=42)

A useful tool for dividing up the data to train and test.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pitch_pure, pitch['pitch_type'], test_size = 0.2)


In [37]:
from sklearn.ensemble import RandomForestClassifier 

forest = RandomForestClassifier(n_estimators=10, max_features=10)
forest.fit(X_train, y_train)
forest.score(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(forest.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(X_test, y_test)))


Accuracy on training set: 0.995
Accuracy on test set: 0.865


87% accuracy on the test set! While I am pretty happy with the performance of the model some things I would incorporate in future models as I deploy them in the real world be to account for the sequence aspect present in real games so that the model can be aware of what the pitcher threw in the past 2 or 3 pitches as pitchers are known to "set up" pitches.

In [38]:
plot_feature_importances_cancer(forest)

NameError: ignored